In [1]:
from ultralytics import YOLO

import cv2


In [ ]:
!pip install roboflow
 
from roboflow import Roboflow

rf = Roboflow(api_key="sPv82OqZiWWjb80CNPiG")

project = rf.workspace("agrobot-dpgqt").project("agrobot-uaaxn")

version = project.version(3)

dataset = version.download("yolov11")

 

In [ ]:

# Load a model
model = YOLO("yolo11n.pt")  

# Train the model with 2 GPUs
results = model.train(
    data=r"/home/sakar04/agrobot_train/Agrobot-3/data.yaml", 
    epochs=100, imgsz=640, 
    device=0,
    batch=0.6,
    resume=False,
    plots=True)

In [ ]:
from ultralytics.data.annotator import auto_annotate

auto_annotate(
    data="/home/sakar04/agrobot_train/dataset",
    det_model="/home/sakar04/agrobot_train/runs/detect/train2/weights/best.pt",
    sam_model="/home/sakar04/agrobot_train/yolo11n.pt",
    device="cuda",
    output_dir="/home/sakar04/agrobot_train/labels",
)

In [ ]:
import os
import numpy as np
from ultralytics.utils.ops import segments2boxes

def convert_segmentations_to_bboxes(input_dir, output_dir):
    """
    Converts segmentation masks in .txt files to bounding boxes and saves them.

    Args:
        input_dir (str): Path to the directory containing segmentation label .txt files.
        output_dir (str): Path to the directory where bounding box files will be saved.
    """
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Step 1: Find all .txt files in the input directory
    txt_files = [os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith('.txt')]
    print(f"Found {len(txt_files)} .txt files in the input directory.")

    # Step 2: Process each file
    for file_path in txt_files:
        with open(file_path, 'r') as file:
            # Load segments from the file
            segments = []
            for line in file:
                points = list(map(float, line.strip().split()))
                # Extract the polygon points only (ignore the first class ID)
                polygon_points = points[1:]
                segments.append(polygon_points)

            # Log segments for debugging
            print(f"Segments in {os.path.basename(file_path)}:")
            print(segments)

        # Step 3: Convert to the required NumPy array format
        try:
            formatted_segments = []
            for segment in segments:
                # Reshape to (-1, 2) where each row is a (x, y) coordinate
                reshaped_segment = np.array(segment).reshape(-1, 2)
                formatted_segments.append(reshaped_segment)
            
            formatted_segments = np.array(formatted_segments, dtype=object)
            
            # Convert segments to bounding boxes
            bboxes = segments2boxes(formatted_segments)
            print(f"Bounding boxes for {os.path.basename(file_path)}:")
            print(bboxes)
        except ValueError as e:
            print(f"Error processing {file_path}: {e}")
            continue

        # Step 4: Save bounding boxes to the output directory
        output_file_path = os.path.join(output_dir, os.path.basename(file_path))
        with open(output_file_path, 'w') as output_file:
            for bbox in bboxes:
                bbox_str = ' '.join(map(str, bbox))
                output_file.write(f"{bbox_str}\n")

if __name__ == "__main__":
    input_dir = "/home/sakar04/agrobot_train/roboflow_data/labels"  # Replace with your path
    output_dir = "/home/sakar04/agrobot_train/labels2"  # Replace with your path

    convert_segmentations_to_bboxes(input_dir, output_dir)


In [ ]:
import os
import numpy as np
from ultralytics.utils.ops import segments2boxes

def convert_segmentations_to_bboxes(input_dir, output_dir):
    """
    Converts segmentation masks in .txt files to bounding boxes and saves them with class IDs.

    Args:
        input_dir (str): Path to the directory containing segmentation label .txt files.
        output_dir (str): Path to the directory where bounding box files will be saved.
    """
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Step 1: Find all .txt files in the input directory
    txt_files = [os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith('.txt')]
    print(f"Found {len(txt_files)} .txt files in the input directory.")

    # Step 2: Process each file
    for file_path in txt_files:
        with open(file_path, 'r') as file:
            segments = []
            class_ids = []

            for line in file:
                points = list(map(float, line.strip().split()))
                if len(points) < 3:
                    print(f"Skipping invalid line in {file_path}: {line.strip()}")
                    continue
                
                # Extract class ID and polygon points
                class_ids.append(int(points[0]))
                polygon_points = points[1:]

                # Validate that polygon points can form (x, y) pairs
                if len(polygon_points) % 2 != 0:
                    print(f"Skipping invalid polygon in {file_path}: {line.strip()}")
                    continue
                
                segments.append(polygon_points)

        # Skip files with no valid data
        if not segments:
            print(f"No valid segments found in {file_path}. Skipping.")
            continue

        # Step 3: Convert to the required NumPy array format
        try:
            formatted_segments = [np.array(segment).reshape(-1, 2) for segment in segments]
            formatted_segments = np.array(formatted_segments, dtype=object)
            
            # Convert segments to bounding boxes
            bboxes = segments2boxes(formatted_segments)
        except Exception as e:
            print(f"Error processing {file_path}: {e}")
            continue

        # Step 4: Combine class IDs with bounding boxes
        output_data = [
            f"{class_id} " + " ".join(map(str, bbox))
            for class_id, bbox in zip(class_ids, bboxes)
        ]

        # Step 5: Save bounding boxes with class IDs to the output directory
        output_file_path = os.path.join(output_dir, os.path.basename(file_path))
        with open(output_file_path, 'w') as output_file:
            output_file.write("\n".join(output_data))

        print(f"Processed and saved: {output_file_path}")

if __name__ == "__main__":
    input_dir = "/home/sakar04/agrobot_train/labels/labels"  # Replace with your path
    output_dir = "/home/sakar04/agrobot_train/labels2"  # Replace with your path

    convert_segmentations_to_bboxes(input_dir, output_dir)
